In [2]:
# import packages
import pandas as pd
import altair as alt
import requests
from ecostyles import EcoStyles

In [3]:
# Creating table based on Lenard Curtis - Rugby Finance Report 2024.pdf - Loss Makers table (pg 30)
# https://cdn.prod.website-files.com/64bd55318b46daf89b6c9e4e/66f3e16b31ad585626beab36_Leonard%20Curtis%20-%20Rugby%20Finance%20Report%202024.pdf
loss_table = pd.DataFrame({
    'club': ['Gloucester Rugby', 'Northampton Saints', 'Leicester Tigers', 'Newcastle Falcons', 'Exeter Chiefs', 'Harlequins', 'Sale Sharks', 'Bath Rugby', 'Bristol Bears', 'Saracens', 'London Irish', 'Wasps', 'Worcester Warriors',
             'Gloucester Rugby', 'Northampton Saints', 'Leicester Tigers', 'Newcastle Falcons', 'Exeter Chiefs', 'Harlequins', 'Sale Sharks', 'Bath Rugby', 'Bristol Bears', 'Saracens', 'London Irish', 'Wasps', 'Worcester Warriors',
             'Gloucester Rugby', 'Northampton Saints', 'Leicester Tigers', 'Newcastle Falcons', 'Exeter Chiefs', 'Harlequins', 'Sale Sharks', 'Bath Rugby', 'Bristol Bears', 'Saracens', 'London Irish', 'Wasps', 'Worcester Warriors',
             'Gloucester Rugby', 'Northampton Saints', 'Leicester Tigers', 'Newcastle Falcons', 'Exeter Chiefs', 'Harlequins', 'Sale Sharks', 'Bath Rugby', 'Bristol Bears', 'Saracens', 'London Irish', 'Wasps', 'Worcester Warriors',
             'Gloucester Rugby', 'Northampton Saints', 'Leicester Tigers', 'Newcastle Falcons', 'Exeter Chiefs', 'Harlequins', 'Sale Sharks', 'Bath Rugby', 'Bristol Bears', 'Saracens', 'London Irish', 'Wasps', 'Worcester Warriors',
             'Gloucester Rugby', 'Northampton Saints', 'Leicester Tigers', 'Newcastle Falcons', 'Exeter Chiefs', 'Harlequins', 'Sale Sharks', 'Bath Rugby', 'Bristol Bears', 'Saracens', 'London Irish', 'Wasps', 'Worcester Warriors'],
    'year': [17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17,
             18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18,
             19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19,
             20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20,
             21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21,
             22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22],
    'amt':  [-2181, -2788, -1236, -4278, 659, -154, -1709, -3149, -7236, -3887, -3492, -9723, 1933,
             5353, 4275, 5218, 3882, 1594, -3482, 3687, 5573, 2845, -5296, 2398, 4624, -4870,
             -982, -1450, -3082, -422, -2505, -3054, -870, -2930, -4115, -8016, -5423, -12444, -1611,
             1040, 960, -1077, 3378, -8002, 928, 403, -162, -3512, -4603, -1966, -7984, None,
             -695, 313, -2230, -2515, -1870, -1575, -3522, -551, -3320, -5120, None, None, None,
             -544, -1064, -1536, -2360, -2983, -3615, -4112, -4429, -4555, -5295, None, None, None]})

In [4]:
loss_table_mean = pd.DataFrame({
    'club': ['Average', 'Average', 'Average', 'Average', 'Average', 'Average'],
    'year': [17, 18, 19, 20, 21, 22],
    'amt':  [loss_table[loss_table['year'] == 17]['amt'].mean(), loss_table[loss_table['year'] == 18]['amt'].mean(),
             loss_table[loss_table['year'] == 19]['amt'].mean(), loss_table[loss_table['year'] == 20]['amt'].mean(),
             loss_table[loss_table['year'] == 21]['amt'].mean(), loss_table[loss_table['year'] == 22]['amt'].mean()]
})

In [6]:
loss_table = pd.concat([loss_table, loss_table_mean], axis=0)

In [5]:
# Create styles instanceß
styles = EcoStyles()
# Register and enable a theme
styles.register_and_enable_theme(theme_name="article")  # or "article"

In [36]:
# Creating altiar chart using rugby loss table
# Determine x-axis range from your data
year_min = loss_table['year'].min()
year_max = loss_table['year'].max()

# Create a dummy DataFrame for the rectangle
red_rect_data = pd.DataFrame({
    'year': [year_min, year_max],
    'y0': [0, 0],
    'y1': [-14000, -14000]  # Adjust to your lowest expected value
})

# 1. Red area as a pseudo-rectangle
red_rect = alt.Chart(red_rect_data).mark_area(
    color='red',
    opacity=0.2
).encode(
    x='year:Q',
    y='y1:Q',
    y2='y0:Q'
)

# Create a dummy DataFrame for the rectangle
green_rect_data = pd.DataFrame({
    'year': [year_min, year_max],
    'y0': [6000, 6000],
    'y1': [0, 0]  # Adjust to your lowest expected value
})

# 1. Red area as a pseudo-rectangle
green_rect = alt.Chart(green_rect_data).mark_area(
    color='green',
    opacity=0.2
).encode(
    x='year:Q',
    y='y1:Q',
    y2='y0:Q'
)

base = alt.Chart(loss_table).encode(x=alt.X('year:Q', axis=alt.Axis(labelExpr=' "20" + datum.value + "/" + (datum.value + 1)')), 
                             y=alt.Y('amt:Q', axis=alt.Axis(labelExpr='"£" + datum.value / 1000 + " bn"'), title='Rugby Finances: profit/loses by club'), 
                             detail=alt.Detail('club:N'),
                             color=alt.condition(alt.datum.club == 'Average', alt.value('black'), alt.value('grey')))

lines = base.mark_line()

annotated_clubs = ['Average', 'Saracens', 'Harlequins', 'Leicester Tigers']
text = base.transform_filter(
    alt.FieldOneOfPredicate(field='club', oneOf=annotated_clubs)
).transform_window(
    rank='rank()', 
    sort=[alt.SortField('year', order='descending')], 
    groupby=['club']
).transform_filter('datum.rank == 1'
).mark_text(
    align='left', dx=5, color='black'
).encode(
    text='club:N'
)

chart = alt.layer(green_rect, red_rect, lines, text).properties(
    title='Rugby Finances Over Time'
)

In [37]:
chart.display()

alt.LayerChart(...)

In [39]:
styles.add_source(chart, 'Leonard Curtis - Rugby Finance Report 2024')

alt.LayerChart(...)

In [40]:
# Save to png
chart.save('rugby_finances_1.png', scale_factor=2)

In [41]:
# Save to json
chart.save('rugby_finances_1.json', scale_factor=2)